# Graph

Solutions to Leetcode problems that use graphs.

In [8]:
from typing import List

import collections

class Node:
    def __init__(self, val = 0, neighbors = None):
        self.val = val
        self.neighbors = neighbors if neighbors is not None else []

# Problem One: Number of Islands (Medium)

[Leetcode #200](https://leetcode.com/problems/number-of-islands/)

- Initialize `visited` set to `False` for each cell (or use a `Set` for this)
- Iterate through each cell. If it is a `1` and it is not visited:
    - Run BFS on cell
    - Increment islands

BFS:
    - Use queue
    - Check exisiting neighbours. If they are `1` and not visited, add to queue and mark as vissited

In [6]:
def numIslands(grid: List[List[str]]) -> int:
        if not grid:
            return 0
        
        rows, cols = len(grid), len(grid[0])
        visited = set()
        islands = 0

        def bfs(r, c):
            q = collections.deque()
            visited.add((r, c))
            q.append((r, c))

            while q:
                row, col = q.popleft()
                dirs = [[1, 0], [-1, 0], [0, 1], [0, -1]]

                for dr, dc in dirs:
                    nr, nc = row + dr, col + dc
                    if (nr in range(rows) and nc in range(cols) and
                        grid[nr][nc] == '1' and (nr, nc) not in visited):
                        q.append((nr, nc))
                        visited.add((nr, nc))
                

        for i in range(rows):
            for j in range(cols):
                if grid[i][j] == '1' and (i, j) not in visited:
                    bfs(i, j)
                    islands += 1
        
        return islands

In [7]:
assert numIslands([["1","1","1","1","0"],["1","1","0","1","0"],["1","1","0","0","0"],["0","0","0","0","0"]]) == 1

# Problem Two: Clone Graph (Medium)

[Leetcode #133](https://leetcode.com/problems/clone-graph/)

- Use dfs
- Create a hashmap mapping old graph nodes to references of the new copies
- DFS:
    - If copy exists in hashmap, return copy
    - Else, create copy and add it to hasmap. For each neighbouring node, recursively call dfs function and add the return value (the copied neighbour) to created copies neighbour list. Return the copy

In [9]:
def cloneGraph(node: 'Node') -> 'Node':
        # Dict to map reference of old nodes to references of new copied nodes
        oldToNew = {}
        
        def dfs(node):
            # If node is in copy hashmap, return the reference to the copy
            if node in oldToNew:
                return oldToNew[node]

            # Create copy reference
            copy = Node(node.val)
            # Create new mapping form old to new copy
            oldToNew[node] = copy

            # Recursively call dfs on all neighbouring nodes
            for n in node.neighbors:
                copy.neighbors.append(dfs(n))  # Add each neighbour copy to this copies neighbours
            
            return copy  # Return the copy
        
        return dfs(node) if node else None  # Begin recursive calls, and return None if None was given

# Problem Three: Max Area of Island (Medium)

[Leetcode #695](https://leetcode.com/problems/max-area-of-island/)

(Basically same as **Problem One** except we count the size of islands and compare to a `maxArea` value)

- Keep track of visited nodes
- Run BFS on each node
    - for each neighbouring `1` node that isn't visited, add `1` to the island size and mark as visited

In [10]:
def maxAreaOfIsland(grid: List[List[int]]) -> int:
        if not grid:
            return 0

        rows, cols = len(grid),len(grid[0])
        visited = set()
        maxArea = 0

        def bfs(r, c):
            q = collections.deque()
            q.append((r, c))
            visited.add((r, c))
            size = 1

            while q:
                row, col = q.popleft()
                dirs = [(1, 0), (-1, 0), (0, 1), (0, -1)]  # Up, down, right, left

                for dr, dc in dirs:
                    nr, nc = row + dr, col + dc
                    if nr in range(rows) and nc in range(cols) and grid[nr][nc] == 1 and (nr, nc) not in visited:
                       q.append((nr, nc))
                       visited.add((nr, nc))
                       size += 1
            
            return size

        for r in range(rows):
            for c in range(cols):
                if grid[r][c] == 1 and (r, c) not in visited:
                    maxArea = max(maxArea, bfs(r, c))
                    visited.add((r, c))

        return maxArea

# Problem Four: Pacific Atlantic Water Flow (Medium)

[Leetcode #417](https://leetcode.com/problems/pacific-atlantic-water-flow/)

- Run DFS from all pacific bordering cells, then all atlantic bordering cells. Whichever nodes were visited by both DFS are reachable by both.

In [11]:
def pacificAtlantic(heights: List[List[int]]) -> List[List[int]]:
        # Resultant array
        res = []
        # Sets of nodes reachable from pacific and atlantic
        pac, atl = set(), set()
        rows, cols = len(heights), len(heights[0])

        def dfs(r, c, pacific):
            """
            Search all nodes starting from a pacific or atlantic ocean bordering node. `pacific` is `True` if we
            are starting the search from the pacific, `False` if we are starting from atlantic
            
            """
            # Initialize stack and add current node
            stack = [(r, c)]
            
            # Helper functions for seeing if node is visited and marking as visited
            visited = lambda nr, nc: (nr, nc) in pac if pacific else (nr, nc) in atl
            visit = lambda nr, nc: pac.add((nr, nc)) if pacific else atl.add((nr, nc))

            # Visit current node
            visit(r, c)

            while stack:
                row, col = stack.pop()
                
                # Look at all neighbours
                dirs = [(1, 0), (-1, 0), (0, 1), (0, -1)]
                for dr, dc in dirs:
                    nr, nc = row + dr, col + dc
                    # If water can come to this node from neighbouring node and it is not visited
                    if nr in range(rows) and nc in range(cols) and heights[nr][nc] >= heights[row][col] and not visited(nr, nc):
                        visit(nr, nc)  # Mark neighbour as visited
                        stack.append((nr, nc))  # Add neighbour to stack

        for r in range(rows):
            # Search which cells can be reached by pacific (c == 0)
            dfs(r, 0, True)
            # Search which cells can be reached by atlantic (c == cols - 1)
            dfs(r, cols - 1, False)

        for c in range(cols):
            # Search which cells can be reached by pacific (r == 0)
            dfs(0, c, True)
            # Search which cells can be reached by atlantic (r == rows - 1)
            dfs(rows - 1, c, False)
        
        # Combine intersection of sets into result array
        # Note: do not use set intersection method since the order of the answers will not be correct (sets have no order)
        for r in range(rows):
            for c in range(cols):
                if (r, c) in pac and (r, c) in atl:
                    res.append((r, c))
        
        return res

# Problem Four: Surrounded Regions (Medium)

[Leetcode #130](https://leetcode.com/problems/surrounded-regions/)

- Go around the border, whichever cell is an `O`, start a DFS from there. This region is NOT surrounded. Change all not surrounded regions to something temporary (e.g. `T`)
- Replace all remaining `O` which are surrounded with `X`
- Replace the temporary cells (e.g. `T`) with `O`

In [12]:
def solve(board: List[List[str]]) -> None:
        """
        Do not return anything, modify board in-place instead.
        """
        ROWS, COLS = len(board), len(board[0])

        def dfs(r, c):
            if r < 0 or c < 0 or r >= ROWS or c >= COLS or board[r][c] != 'O':
                return
            
            board[r][c] = 'T'
            dfs(r - 1, c)
            dfs(r + 1, c)
            dfs(r, c - 1)
            dfs(r, c + 1)
        
        # Replace unsurrounded O w/ T
        for r in range(ROWS):
            for c in range(COLS):
                if board[r][c] == 'O' and r in [0, ROWS-1] or c in [0, COLS-1]:
                    dfs(r, c)
        
        # Replace all remainig (surrounded) O with X and replace all T with O
        for r in range(ROWS):
            for c in range(COLS):
                if board[r][c] == 'O':
                    board[r][c] = 'X'
                elif board[r][c] == 'T':
                    board[r][c] = 'O'

# Problem Five: Rotting Oranges (Medium)

[Leetcode #994](https://leetcode.com/problems/rotting-oranges/)

- Iterate to find number of fresh fruit and to start filling the queue with the initial "wave" of rotten fruits
- BFS on each rotting orange
- Each iteration, pop the whole queue and add 1 to time
- For each node in the queue, visit fresh neighbours in next iteration. Also make this node rotten and decrement fresh count


In [13]:
def orangesRotting(grid: List[List[int]]) -> int:
        ROWS, COLS = len(grid), len(grid[0])
        q = collections.deque()
        time, fresh = 0, 0
        dirs = [(1, 0), (-1, 0), (0, 1), (0, -1)]

        for r in range(ROWS):
            for c in range(COLS):
                if grid[r][c] == 1:
                    fresh += 1
                if grid[r][c] == 2:
                    q.append((r, c))
        
        while q and fresh > 0:
            for i in range(len(q)):
                r, c = q.popleft()

                for dr, dc in dirs:
                    nr, nc = r + dr, c + dc
                    if nr in range(ROWS) and nc in range(COLS) and grid[nr][nc] == 1:
                        grid[nr][nc] = 2
                        q.append((nr, nc))
                        fresh -= 1
            
            time += 1
        
        return time if fresh == 0 else -1

In [14]:
assert orangesRotting([[2,1,1],[1,1,0],[0,1,1]]) == 4

# Problem Six: Walls and Gates (Medium)

[Leetcode #286](https://leetcode.com/problems/walls-and-gates/)

- Start by adding all gates to the queue, and initializing visited set
- Set `dist` to `0`
- While queue is full:
    - For all items currently in queue:
        - Set distance to `dist` (won't affect gates on first pass since we start at 0)
        - For each neighbour:
            - If in bounds, is not visited, and is not a gate or wall:
                - Add to queue to be updated next iteration
                - Mark as visited
    - Increment `dist`

In [15]:
def wallsAndGates(rooms: List[List[int]]) -> None:
        """
        Do not return anything, modify rooms in-place instead.
        """
        # Get ROWS and COLS
        ROWS, COLS = len(rooms), len(rooms[0])
        # Initialize queue and visited set
        q = collections.deque()
        visited = set()
        
        # Add gates to queue
        for r in range(ROWS):
            for c in range(COLS):
                if rooms[r][c]== 0:
                    q.append((r, c))
                    visited.add((r, c))
        
        # Keep track of how many gates we have
        dist = 0
        while q:
            for i in range(len(q)):
                r, c = q.popleft()
                rooms[r][c] = dist

                dirs = [(1, 0), (-1, 0), (0, 1), (0, -1)]
                for dr, dc in dirs:
                    nr, nc = r + dr, c + dc
                    if nr in range(ROWS) and nc in range(COLS) and (nr, nc) not in visited and rooms[nr][nc] > 0:
                        visited.add((nr, nc))
                        q.append((nr, nc))


            dist += 1

# Problem Seven: Course Schedule (Medium)

[Leetcode #207](https://leetcode.com/problems/course-schedule/)

**Key point**: Graph must be a ***DAG*** (Directed _acyclic_ graph), i.e. if there is a cycle, the schedule cannotbe finished, else, it can be finished

- Create hashmap mapping courses to their prerequisites
- Run a DFS on each course
    - If we revisit a course, cycle detected, return false
    - If course has no prerequisites return true
    - Mark course as visited
    - Visit all prequisite forces. Make recursive DFS calls. If any calls return false, then this course cannot be completed either. Return false.
    - If all recursive calls are succesful, remove this course from visited. This is because another course might have this as a prereq and we don't want to think this was a cycle. Also set prerequisites to none for quick comparison to see course can be completed in future calls. Return true.
    

In [16]:
def canFinish(numCourses: int, prerequisites: List[List[int]]) -> bool:
        # Initialize prereq hashmap and visited set 
        prereq = { i:[] for i in range(numCourses)}  # Maps a course to its prerequisites
        visited = set()
        
        # Fill prereq based on given prerequisites
        for req in prerequisites:
            a, b = req
            prereq[a].append(b)
        
        def dfs(course):
            # If course was already visited, cycle exists
            if course in visited:
                return False
            
            # If no prereqs, it can be finished
            if prereq[course] == []:
                return True
            
            # Add course to visited
            visited.add(course)
            # Run DFS on each prerequisite
            for c in prereq[course]:
                if not dfs(c):  # If any of the DFS return false, cannot finish this course
                    return False
            
            # Remove this course from visited incase other courses also visit this course and their path ends here
            visited.remove(course)
            # Empty out this course so it can return True from now on
            prereq[course] = []
            # Return true since this course can be finished
            return True
        
        # Run our dfs on each course
        for i in range(numCourses):
            if not dfs(i):  # If any return false, we cannot finish this course schedule
                return False
        
        return True

# Problem Eight: Course Schedule II (Medium)

[Leetcode #210](https://leetcode.com/problems/course-schedule-ii/)

- Initialize hashmap which maps each course to its prereqs
- Initialize output to empty list
- Intialize set of visited nodes. These nodes are nodes that have been added to the output list. Initialize set of current nodes on the path.
- Fill prereqs hashmap
- Run DFS on each course:
    - If course was in current path, cycle exists so return false
    - If course was visited, it was alr in output so return true
    - Add course to current path
    - Run DFS on each prereq. Return false if any calls return false (cycle was detected)
    - Remove course from current path
    - Add course to visited and output lists. Return true

In [17]:
def findOrder(numCourses: int, prerequisites: List[List[int]]) -> List[int]:
        # Initialize prereq hashmap which maps a course to its prerequisites
        prereq = { i: [] for i in range(numCourses) }
        visited, cycle = set(), set()  # Set of visited nodes (in output already) and nodes on current path
        res = []  # Output array
        
        # Fill prerequisites array
        for a, b in prerequisites:
            prereq[a].append(b)

        def dfs(i):
            # If node was visited in this path, cycle exists, return False
            if i in cycle:
                return False
            
            # If this node was visited and added to output, return True
            if i in visited:
                return True
            
            # Add current node to path
            cycle.add(i)
            # Visit all prerequisites
            for pre in prereq[i]:
                if not dfs(pre):  # If any of the prereqs had a cycle, return False
                    return False
            
            # This node is no longer in the current path we are examining
            cycle.remove(i)
            # Add this node to visisted list and output list
            visited.add(i)
            res.append(i)
            return True
            
        # Run the dfs on all the courses
        for i in range(numCourses):
            if not dfs(i):  # If any of the searches return False (find a cycle) return empty list
                return []

        return res

# Problem Nine: Redundant Connection (Medium)

[Leetcode #684](https://leetcode.com/problems/redundant-connection/)

**Key concept**: Use Union-Find. If we run `union(a, b)` on two nodes `a` and `b` with the same parents, edge `[a, b]` is the redundant edge

- Initialize `parent` list with all nodes as parents of themselves
- Initialize all nodes to `rank` 1
- For each edge `[a, b]`:
    - If `union(a, b)` returns false, return `[a, b]`

In [19]:
def findRedundantConnection(edges: List[List[int]]) -> List[int]:
        # Initialize each node to be its own parent
        parent = [i for i in range(len(edges) + 1)]
        # Initialize all ranks to 1
        rank = [1] * (len(edges) + 1)

        # Find parent of node n
        def find(n):
            p = parent[n]

            while p != parent[p]:
                p = parent[parent[p]]  # Path compression to shorten links
                p = parent[p]
            
            return p

        # Merge n1 and n2. Return false if they had the same parent
        def union(n1, n2):
            p1, p2 = find(n1), find(n2)

            if p1 == p2:
                return False
            
            if rank[p1] > rank[p2]:
                parent[p2] = p1
                rank[p1] += rank[p2]
            else:
                parent[p1] = p2
                rank[p2] += rank[p1]
            
            return True
        
        # Return the edge that returned false on union operation
        for a, b in edges:
            if not union(a, b):
                return [a, b]

# Problem Ten: Number of Connected Components in an Undirected Graph (Medium)

[Leetcode #323](https://leetcode.com/problems/number-of-connected-components-in-an-undirected-graph/)

- Run DFS on each node if not visited:
    - In DFS nodes get marked as visited
    - Increment connected

In [20]:
def countComponents(self, n: int, edges: List[List[int]]) -> int:
        # Initialize adjacency matrix and visisted set
        adj = collections.defaultdict(list)
        visited = set()
        # Fill adjacency matrix. Undirected graph so fill both ways
        for a, b in edges:
            adj[a].append(b)
            adj[b].append(a)
        
        # Initialize number of connected components to 0
        connected = 0

        def dfs(i):
            # If we have visited this node before, return
            if i in visited:
                return
             
            # Add node to visited
            visited.add(i)
            
            # Visit all adjacent nodes
            for n in adj[i]:
                dfs(n)
        
        # For each node
        for i in range(n):
            # If not visited yet
            if i not in visited:
                dfs(i)  # Run a DFS from this node to find all nodes connected to this component
                connected += 1  # Increment number of connected components
        
        return connected

# Problem Eleven: Graph Valid Tree (Medium)

[Leetcode #261](https://leetcode.com/problems/graph-valid-tree/)

- Tree if no cycle and all components are connected
- Run DFS on any node (e.g. 0). If cycle is found return false. Mark each node as visited. If we visit `n` nodes, return true. Else, return false.

In [22]:
def validTree(n: int, edges: List[List[int]]) -> bool:
        # Return true for emptyt graph
        if not n:
            return True
        
        # Build adjacency matrix. Undirected so edges go  both ways
        adj = collections.defaultdict(list)
        for a, b in edges:
            adj[a].append(b)
            adj[b].append(a)
        
        # Initialize visited set
        visited = set()

        def dfs(i, prev):
            # If node was visited before, cycle found. Return false
            if i in visited:
                return False
            
            # Add node to visited
            visited.add(i)
            
            # Visit all adjacent nodes
            for n in adj[i]:
                if n != prev:  # Graph is undirected so don't visit the previous (already visited) node (avoid false negative)
                    if not dfs(n, i):  # Run DFS recursively, return False if any of the adjacent ndoes had a cycle
                        return False
            
            return True  # Return true if all adjacent nodes were succesfuly visited
        
        # Run DFS on first node. If cycle was found return False
        if not dfs(0, None):
            return False
        
        # Return True iff number of visited nodes is n
        return len(visited) == n

# Problem Twelve: Word Ladder (*Hard*)

[Leetcode #127](https://leetcode.com/problems/word-ladder/)

- Create adjacency matrix which maps patterns to the words found in our wordlist. E.g `*ot` maps to `lot`, `dot` and `hot` given `wordList=["hot","dot","dog","lot","log","cog"]`
- Run BFS in this graph to find shortest path from `beginWord` to `endWord` node

In [24]:
def ladderLength(beginWord: str, endWord: str, wordList: List[str]) -> int:
        # Return 0 if the endWord is not in the list
        if endWord not in wordList:
            return 0
        
        # Initialize adjacency matrix
        adj = collections.defaultdict(list)
        
        # Add the begin word to the word list
        wordList.append(beginWord)
        # Create patterns of all words and add the word to the list of matches of the pattern
        for word in wordList:
            for j in range(len(word)):
                pattern = word[:j] + '*' + word[j+1:]
                adj[pattern].append(word)
        
        # Initialize visited set and queue to contain the beignWord
        visited = set([beginWord])
        q = collections.deque([beginWord])
        res = 1  # Initialize result to 1
        while q:
            # Loop through all items currently in queue
            for i in range(len(q)):
                word = q.popleft()
                
                # Return res if we found the word
                if word == endWord:
                    return res
                
                # Create pattern from the word to get adjacent words
                for j in range(len(word)):
                    pattern = word[:j] + '*' + word[j+1:]
                    for nei in adj[pattern]:
                        if nei not in visited:
                            visited.add(nei)
                            q.append(nei)

            res += 1  # Increment result

        return 0  # If we get to this line, no path was found

# Problem Thirteen: Network Delay Time (*Hard*)

[Leetcode #743](https://leetcode.com/problems/network-delay-time/)

**Key concept**: Dijkstra!

- Use `minHeap` with `(weight, node)` where min `weight` is returned
- Create adj matrix
- Initialize min heap at starting node with weight `0`
- Initialize visited set
- Initialize time to `0`
- While minHeap is full:
    - Pop min
    - If node is visited, continue
    - Mark node as visited, update time to weight of node
    - For all adjacent nodes:
        - If not visited, push the node to heap
- Return time if all nodes visited, else `-1`

In [25]:
def networkDelayTime(times: List[List[int]], n: int, k: int) -> int:
        # Create adj matrix
        edges = collections.defaultdict(list)
        for u, v, w in times:
            edges[u].append((v, w))
        
        # Initialize min heap with starting node and a visited set
        minHeap = [(0, k)]
        visited = set()

        # Initialize time
        t = 0

        while minHeap:
            w1, n1 = heapq.heappop(minHeap)
            
            # If node already visited, continue
            if n1 in visited:
                continue
            
            # Visit node
            visited.add(n1)

            # Update t
            t = w1

            # Visit neighbors and add to min heap
            for n2, w2 in edges[n1]:
                if n2 not in visited:
                    heapq.heappush(minHeap, (w1 + w2, n2))
        
        return t if len(visited) == n else -1